In [1]:
import os
import sys
import pandas as pd

current_path = os.getcwd()
root_path = os.path.abspath(os.path.join('../..'))

print(root_path)

C:\Users\guyi\Documents\projects\great_expectations\src


In [2]:
# util and data_sanity_check folders arein the same level
util_path = os.path.abspath(os.path.join('../util/'))

print(util_path)

C:\Users\guyi\Documents\projects\great_expectations\src\data_validation\util


In [3]:
if util_path not in sys.path:
    sys.path.append(util_path)

In [4]:
from project_config import get_project_config_local

In [5]:
## Data and great expectations folders should be created in order to run this script

data_dir = os.path.join(root_path, "data") 
print(data_dir)

expectations_store_dir = os.path.join(root_path, "great_expectations", "expectations")
print(expectations_store_dir)

validations_store_dir = os.path.join(root_path, "great_expectations", "validations")
print(validations_store_dir)

checkpoint_store_dir = os.path.join(root_path, "great_expectations", "checkpoints")
print(checkpoint_store_dir)

local_site_datadoc_dir = os.path.join(root_path, "great_expectations", "uncommitted/data_docs/local_site/")
print(local_site_datadoc_dir)

## project configuration let great expectations know the folder structure and locations

project_config = get_project_config_local(expectations_store_dir, validations_store_dir, checkpoint_store_dir, local_site_datadoc_dir)
print(project_config)

C:\Users\guyi\Documents\projects\great_expectations\src\data
C:\Users\guyi\Documents\projects\great_expectations\src\great_expectations\expectations
C:\Users\guyi\Documents\projects\great_expectations\src\great_expectations\validations
C:\Users\guyi\Documents\projects\great_expectations\src\great_expectations\checkpoints
C:\Users\guyi\Documents\projects\great_expectations\src\great_expectations\uncommitted/data_docs/local_site/
{
  "anonymous_usage_statistics": {
    "enabled": true,
    "data_context_id": "ba344f21-c1e5-4fe9-af77-1423a7a1a862",
    "usage_statistics_url": "https://stats.greatexpectations.io/great_expectations/v1/usage_statistics",
    "explicit_url": false,
    "explicit_id": false
  },
  "checkpoint_store_name": "checkpoint_store",
  "config_version": 3,
  "data_docs_sites": {
    "local_site": {
      "class_name": "SiteBuilder",
      "show_how_to_buttons": "true",
      "store_backend": {
        "class_name": "TupleFilesystemStoreBackend",
        "base_directory

In [6]:
from source_config_local_data import get_runtime_datasource_config

datasource_name = "fs_datasource"
execution_engine = "PandasExecutionEngine"

datasource_config = get_runtime_datasource_config(datasource_name, execution_engine)

In [7]:
from ruamel import yaml
from great_expectations.data_context import BaseDataContext

curr_context = BaseDataContext(project_config=project_config)
curr_context.test_yaml_config(yaml.dump(datasource_config))

Attempting to instantiate class from config...
	Instantiating as a Datasource, since class_name is Datasource
	Successfully instantiated Datasource


ExecutionEngine class name: PandasExecutionEngine
Data Connectors:
	default_runtime_data_connector_name:RuntimeDataConnector

	Available data_asset_names (0 of 0):
		Note : RuntimeDataConnector will not have data_asset_names until they are passed in through RuntimeBatchRequest

	Unmatched data_references (0 of 0): []



C:\Users\guyi\Miniconda3\envs\build-test-env\lib\site-packages\great_expectations\datasource\data_connector\runtime_data_connector.py:133: DeprecationWarning: Specifying batch_identifiers as part of the RuntimeDataConnector config is deprecated as of v0.15.1 and will be removed by v0.18. Please configure batch_identifiers as part of Assets instead.
  warnings.warn(


In [8]:
curr_context.add_datasource(**datasource_config)

In [9]:
import datetime
from great_expectations.core.batch import RuntimeBatchRequest

current_path = os.getcwd()
data_path = os.path.join(current_path, "../../../data")
csv_path = os.path.join(data_path, "data.csv")
df = pd.read_csv(os.path.join(data_dir, csv_path))
df.head()

,Unnamed: 0,Name,Age,Work Exp,Sex,New Employee,Vocation Days
0,0,Tom,20,0,Male,True,5
1,1,Nick,30,4,Male,False,20
2,2,Juli,24,2,Female,False,16


In [10]:
batch_request = RuntimeBatchRequest(
    datasource_name="fs_datasource",
    data_connector_name="default_runtime_data_connector_name",
    data_asset_name="data_asset_name",
    runtime_parameters={"batch_data": df},  # df is your dataframe
    batch_identifiers={"default_identifier_name": "default_identifier"},
)

In [11]:
from expectations import get_data_expectations_column

expectation_suite_name = "check_data"
expectation_suite = get_data_expectations_column(curr_context, expectation_suite_name)

In [12]:
from checkpoint_config import get_checkpoint_config

check_point_name = "checkpoint"

checkpoint_config = get_checkpoint_config(check_point_name, expectation_suite_name)

In [13]:
curr_context.add_checkpoint(**checkpoint_config)
print("check point " + check_point_name +" created ...")

curr_context.test_yaml_config(yaml.dump(checkpoint_config))

check point checkpoint created ...
Attempting to instantiate class from config...
	Instantiating as a SimpleCheckpoint, since class_name is SimpleCheckpoint
	Successfully instantiated SimpleCheckpoint


Checkpoint class name: SimpleCheckpoint
Your current Checkpoint configuration has an empty or missing "validations" attribute.  This
means you must either update your Checkpoint configuration or provide an appropriate validations
list programmatically (i.e., when your Checkpoint is run).
                    


{
  "action_list": [
    {
      "name": "store_validation_result",
      "action": {
        "class_name": "StoreValidationResultAction"
      }
    },
    {
      "name": "store_evaluation_params",
      "action": {
        "class_name": "StoreEvaluationParametersAction"
      }
    },
    {
      "name": "update_data_docs",
      "action": {
        "class_name": "UpdateDataDocsAction",
        "site_names": []
      }
    }
  ],
  "batch_request": {},
  "class_name": "Checkpoint",
  "config_version": 1.0,
  "evaluation_parameters": {},
  "expectation_suite_name": "check_data",
  "module_name": "great_expectations.checkpoint",
  "name": "checkpoint",
  "profilers": [],
  "runtime_configuration": {},
  "validations": []
}

In [14]:
results = curr_context.run_checkpoint(
    checkpoint_name=check_point_name,
    validations=[
        {"batch_request": batch_request},
    ],
)

res = results.to_json_dict()
print(res)

Calculating Metrics:   0%|          | 0/27 [00:00<?, ?it/s]

C:\Users\guyi\Miniconda3\envs\build-test-env\lib\site-packages\jinja2\environment.py:1088: DeprecationWarning: 'soft_unicode' has been renamed to 'soft_str'. The old name will be removed in MarkupSafe 2.1.
  return concat(self.root_render_func(self.new_context(vars)))


{'run_id': {'run_time': '2022-09-16T11:20:44.661107+00:00', 'run_name': None}, 'run_results': {'ValidationResultIdentifier::check_data/__none__/20220916T112044.661107Z/b8289541e16947af3bdf523c66affb37': {'validation_result': {'results': [{'expectation_config': {'kwargs': {'column': 'Name', 'batch_id': 'b8289541e16947af3bdf523c66affb37'}, 'expectation_type': 'expect_column_to_exist', 'meta': {}}, 'success': True, 'result': {}, 'exception_info': {'raised_exception': False, 'exception_traceback': None, 'exception_message': None}, 'meta': {}}, {'expectation_config': {'kwargs': {'column': 'Name', 'batch_id': 'b8289541e16947af3bdf523c66affb37'}, 'expectation_type': 'expect_column_values_to_not_be_null', 'meta': {}}, 'success': True, 'result': {'element_count': 3, 'unexpected_count': 0, 'unexpected_percent': 0.0, 'partial_unexpected_list': [], 'partial_unexpected_index_list': None, 'partial_unexpected_counts': []}, 'exception_info': {'raised_exception': False, 'exception_traceback': None, 'ex